In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import plotly
np.random.seed(0)
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../input/winequalityred/winequality-red.csv') # ,sep=';')
# Suppose the target is a binary variable
for i in range(len(df['quality'])):
    if df['quality'][i]<=4:
        df['quality'][i]='Low'
    elif df['quality'][i]<=6:
        df['quality'][i]='Medium'
    else:
        df['quality'][i]='High'
df.head()

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df['quality']= label_encoder.fit_transform(df['quality'])
classes=label_encoder.classes_
labels=[0,1,2]
d=dict(zip(labels,classes))
print(d)

In [ ]:
df['quality'].value_counts()

In [ ]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
Y=df['quality']
X =  df[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y.values,random_state=10, test_size = 0.2)
model = RandomForestClassifier(max_depth=6, random_state=0, n_estimators=10)
model.fit(X_train, Y_train)
print(model.feature_importances_)

In [ ]:
import shap
shap_values = shap.TreeExplainer(model).shap_values(X_train)
for which_class in range(0,3):
    shap.summary_plot(shap_values[which_class], X_train)
#{0: 'High', 1: 'Low', 2: 'Medium'}

In [ ]:
kc2=[]
def ABS_SHAP(df_shap,df):
    global kc2
    #import matplotlib as plt
    # Make a copy of the input data
    shap_v = pd.DataFrame(df_shap)
    feature_list = df.columns
    shap_v.columns = feature_list
    df_v = df.copy().reset_index().drop('index',axis=1)
   
    # Determine the correlation in order to plot with different colors
    corr_list = list()
    for i in feature_list:
        b = np.corrcoef(shap_v[i],df_v[i])[1][0]
        corr_list.append(b)
    corr_df = pd.concat([pd.Series(feature_list),pd.Series(corr_list)],axis=1).fillna(0)
    # Make a data frame. Column 1 is the feature, and Column 2 is the correlation coefficient
    corr_df.columns  = ['Variable','Corr']
    corr_df['Sign'] = np.where(corr_df['Corr']>0,'red','blue')
   
    # Plot it
    shap_abs = np.abs(shap_v)
    k=pd.DataFrame(shap_abs.mean()).reset_index()
    k.columns = ['Variable','SHAP_abs']
    kc2 = k.merge(corr_df,left_on = 'Variable',right_on='Variable',how='inner')
    k2 = kc2.sort_values(by='SHAP_abs',ascending = True)
    print(k2)
    colorlist = k2['Sign']
    ax = k2.plot.barh(x='Variable',y='SHAP_abs',color = colorlist, figsize=(5,6),legend=False)
    ax.set_xlabel("SHAP Value (Red = Positive Impact)")

In [ ]:
for which_class in range(0,3):
    ABS_SHAP(shap_values[which_class],X_train)

In [ ]:
print(kc2)

In [ ]:
idx=0
test_row_predict=model.predict(X.iloc[[idx]])
test_row_predict_proba=model.predict_proba(X.iloc[[idx]])
print(test_row_predict)
print(test_row_predict_proba)

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X.iloc[[idx]])
shap.initjs()
for which_class in range(0,3):
    display(shap.force_plot(explainer.expected_value[which_class], shap_values[which_class], X.iloc[[idx]]))

# Additive_nature of shap
# Base_value+sum(shap_values)==predicted_probability

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test.iloc[[0]])
shap_values

In [ ]:
rand_proba=model.predict_proba(X_test.iloc[[0]])
rand_proba

In [ ]:
import math
shap_sum = explainer.expected_value + [i.sum() for i in shap_values]
shap_sum
#assert(math.isclose(shap_sum,rand_proba[0][1],rel_tol=1e-06)) #if this was False, and error would be thrown